<a href="https://colab.research.google.com/github/ghopper3/ChatGPT-Project/blob/main/Copy_of_Fine_Tuning_GPT_3_5_for_Accounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning GPT 3.5 for Finance & Accounting

Large language models like ChatGPT have the potential to be powerful tools for finance and accounting professionals. With the broad set of data they are trained on, they are incredibly powerful generalists that can answer questions on data ranging from atoms to zoology. This is great when you need a generalist, but when you need an industry or company expert it would be great to dial in some of that expertise and increase the model's focus to suit your needs.

By fine-tuning GPT-3.5 on a dataset of finance and accounting questions and answers (in this case, sample questions from the CPA and CFA exams), we can improve the model's ability to generate accurate and informative answers to these types of questions. We can also ensure that the model answers in the appropriate style, which is important for professional settings.

Potential Applications:

The fine-tuned GPT-3.5 model could be used for a variety of applications, such as:
*   Generating answers to finance and accounting questions from users.
*   Creating educational materials for finance and accounting students and professionals.
*   Assisting finance and accounting professionals with their work, such as generating reports and presentations.

For full details on fine-tuning GPT-3.5, please see OpenAI's documentation [here](https://openai.com/blog/gpt-3-5-turbo-fine-tuning-and-api-updates).

# Project Approach:

Collect a dataset of finance and accounting questions and answers.

1. Split the dataset into training and validation sets (80/20).
2. Fine-tune GPT-3.5 on the training set using the following steps:
3. Prompt GPT-3.5 with a question from the training set.
  * Generate an answer to the question.
  * Compare the generated answer to the correct answer.
  * Update the GPT-3.5 parameters to minimize the difference between the generated answer and the correct answer.
4. Evaluate the fine-tuned model on the validation and test sets.


### Before you begin

You'll need to get an [OpenAI API key](https://platform.openai.com/account/api-keys).


*With special thanks to [Sophia Yang](https://www.youtube.com/@SophiaYangDS) and her excellent YouTube tutorial on how to fine-tune GPT-3.5.*

Before we start building our chatbot, we need to install some Python libraries. Here's a brief overview of what each library does:

- **openai**: This is the official OpenAI Python client. We'll use it to interact with the OpenAI API and generate responses for our chatbot.
- **datasets**: This library provides a vast array of datasets for machine learning. We'll use it to load our knowledge base for the chatbot.

Here's how to install those:

In [ ]:
!pip install openai datasets

Next we'll load our dataset of exam questions. We will be uploading a CSV file that contains two columns: one for request (question) and one that contains the response (answer). We'll also split the data into training, testing, and validation sets. (Note: the CSV format didn't work, so we had to go back and convert our CSV to a JSON object.)

In [ ]:
import json

def load_dataset_from_csv(path):
  with open(path, "r", newline="") as f:
    reader = csv.reader(f)
    data = []
    for row in reader:
      data.append(row)

  # Convert the data to a JSON object
  json_data = []
  for row in data:
    json_row = {}
    json_row["request"] = row[1]
    # Check the length of the row list before accessing the row[2] element
    if len(row) > 2:
      json_row["response"] = row[2]
    json_data.append(json_row)

  return json_data

# Load the dataset from the CSV file
json_data = load_dataset_from_csv("FILE_PATH_TO_YOUR_TRAINING_SET")

# Save the JSON data to a file
with open("FILE_PATH_TO_WHERE_YOU_WANT_TO_SAVE_YOUR_JSON_FILE/sample_cpa_cfa_exam_questions.json", "w") as f:
  json.dump(json_data, f)


Let's check the length of our list to make sure all of our records were imported.

In [ ]:
len(ds)

Let's check our headers.

In [ ]:
ds[0]

We initially did some error checking to make sure the data imported correctly. The code is commented out here, but you can uncomment it to check for yourself. We then make sure everything is formatted correctly.

In [ ]:
# print("Type of ds:", type(ds))  # Should be <class 'list'>
# print("Sample element:", ds[0])  # Should be a dictionary

ds_formatted = []
for x in ds:
    print("Type of x:", type(x))  # Should be <class 'dict'>

ds_formatted = []
for x in ds[1:]:  # Skip the header row
    ds_formatted.append({
        "messages": [
            {"role": "system", "content": "You are an accounting and finance expert. If you don't know an answer, don't make one up. Answer as clearly and accurately as possible."},
            {'role': 'user', 'content': x[0]},
            {'role': 'assistant', 'content': x[1] if len(x) > 1 else None}
        ]
    }). #Change the instructions above to meet your particular fine-tuning needs.


In [ ]:
ds_formatted[0]

Next we're ready to split our data into training and validation sets. We shuffle the data first so that both sets contain questions from the CPA and CFA exams.

In [ ]:
import random
random.shuffle(ds_formatted)

In [ ]:
ds_train = ds_formatted[:80]
ds_val = ds_formatted[20:]

In [ ]:
import json

with open('train.jsonl', 'w') as f:
    for line in ds_train:
        json.dump(line, f)
        f.write('\n')

with open('val.jsonl', 'w') as f:
    for line in ds_val:
        json.dump(line, f)
        f.write('\n')

In [ ]:
ls

Now let's check the data.

In [ ]:
!head -n 2 train.jsonl

In [ ]:
!head -n 2 val.jsonl

# Upload data

Now everything is ready to be uploaded to ChatGPT to fine tune our model - split into our training and validation datasets.
NOTE: Insert your OpenAI API key in the spot indicated below.

In [ ]:
import openai

openai.api_key = 'YOUR_API_KEY'


In [ ]:
# Upload training data
train = openai.File.create(
  file=open("train.jsonl", "rb"),
  purpose='fine-tune'
)
train

In [ ]:
train_id = train['id']

In [ ]:
# Upload validation data
val = openai.File.create(
  file=open("val.jsonl", "rb"),
  purpose='fine-tune'
)
val

In [ ]:
val_id = val['id']

# Fine-tuning

With our data cleaned, formatted and loaded, we're ready to fine tune our model.

In [ ]:
# Create a fine-tuned model
response = openai.FineTuningJob.create(
    training_file=train_id,
    validation_file=val_id,
    model="gpt-3.5-turbo"
    )
response

In [ ]:
job_id = response['id']

Let's check the fine tuning status.

In [ ]:
# Retrieve the state of a fine-tune
response = openai.FineTuningJob.retrieve(job_id)
response

It takes a few minutes (depending on the amount of data in your fine tuning set). The model is still training. Let's wait a couple of minutes and try again.

In [ ]:
# Retrieve the state of a fine-tune
response = openai.FineTuningJob.retrieve(job_id)
response

OK, that's done. Now we can look at the results.

In [ ]:
# List up to 10 events from a fine-tuning job
response = openai.FineTuningJob.list_events(id=job_id, limit=10)
response

Now let's check our training loss for a sample of the epochs. The loss decrease wasn't perfect, but it seldom is.

In [ ]:
events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Now let's get the details about our job.

In [ ]:
response = openai.FineTuningJob.retrieve(job_id)
response

In [ ]:
model_id = response["fine_tuned_model"]

# Inference

Great. Now we can test the fine-tuned model.

In [ ]:
test_messages = [
    {"role": "system", "content": "You are an accounting and finance expert. If you don't know an answer, don't make one up. Answer as clearly and accurately as possible."},
    {"role": "user", "content": "What is the accounting equation?"}
  ]

In [ ]:
completion = openai.ChatCompletion.create(
  model=model_id,
  messages=test_messages
)


In the next code block we'll see the answer from our tuning data.

In [ ]:
print(completion.choices[0].message)

In [ ]:
completion1 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=test_messages
)


And now a more detailed answer.

In [ ]:
print(completion1.choices[0].message)

And we're done. Now we can use this fine-tuned model in [Part 2](https://colab.research.google.com/drive/1lmCbmuOssvVCTBCeKuTMpagDSSNZoXM_?usp=sharing), where we will use Retrieval Augmented Generation (RAG) to further improve the model.